In [328]:
# Подключаем необходмые библиотеки и параметры запросов

import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from datetime import datetime
from datetime import timedelta
from matplotlib.ticker import FixedLocator
from IPython.core.display import display, HTML
from math import trunc
import calendar
import warnings
import re
from IPython.core.display import display, HTML
import gzip

warnings.filterwarnings("ignore")

params = {}
params['access_token'] = 'ya29.a0ARrdaM9cmx8d75i7PNk_SOv1ZfOtOANXDvy8_093-NphTObyEaMTNYI7DucQL1JvjvrezWaeOQPSzoq4njkzvWsOQxeEVW48zcfU33paAIRmFG0EHysnZ6hhxEh4zt7DHATCrb48tbI4j9Ao5WcOOJ6S-eiKJFo'
params['ids'] = 'ga:87746406'

header = {'Authorization': 'OAuth AQAAAAAouK-mAAcxxVCgzk_ZQklDlEd30hrpYI0'}
jm_api = '5d5a5620d04c4850426b0076b9aecd14'

In [329]:
def getJMclust(data):
    _url_ = 'https://api.just-magic.org/api_v1.php'

    request_ = {
        'action': 'put_task',
        'apikey': jm_api,
        'task': 'grp_onl',
        'ya_lr': 213,
        'ya_lrws': 213,
        'data': data,
        'f_temakl': 1,
        'f_is_geo': 1,
        'f_is_comm': 1,
        's_qv': 1,
        'domain': 'eldorado.ru'
    }
    while True:
        response = requests.post(_url_, data=request_)

        if response.json()['tid']:
            _request_ = {
                'action': 'get_task',
                'apikey': jm_api,
                'tid': response.json()['tid'],
                'mode': 'csv'
            }

            while True:
                response_ = requests.post(_url_, data=_request_)
                header = 'Content-disposition'

                if header in response_.headers:
                    with open('result.csv.gz', 'wb') as file:
                         file.write(response_.content)

                    df = pd.read_csv('result.csv.gz', compression='gzip', sep='\t')
                    break
            
            break
        
    
    return df

In [330]:
urls_df = pd.read_excel('decreasing_urls.xlsx')

for url in urls_df['url']:
    start_date = datetime.strftime(datetime.today() - timedelta(days=183), '%Y-%m-%d')
    end_date = datetime.strftime(datetime.today() - timedelta(days=153), '%Y-%m-%d')
    start_date_now = datetime.strftime(datetime.today() - timedelta(days=30), '%Y-%m-%d')
    ya_param = {}
    ya_param['date1_a'] = start_date
    ya_param['date2_a'] = end_date
    ya_param['date1_b'] = start_date_now
    ya_param['date2_b'] = 'yesterday'
    ya_param['id'] = 1937671
    ya_param['metrics'] = 'ym:s:visits'
    ya_param['filters'] = "ym:s:lastsignTrafficSource=='organic' AND ym:s:startURL=@'"+url.replace('https://eldorado.ru', '')+"' AND ym:s:lastsignSearchPhrase!*'*эльд*'"
    ya_param['dimensions'] = 'ym:s:lastsignSearchPhrase,ym:s:startURL'
    ya_param['accuracy'] = 'full'
    ya_param['limit'] = 100000

    url_ = 'https://api-metrika.yandex.net/stat/v1/data/comparison'

    response_ = requests.get(url_, params=ya_param, headers=header)
    
    queries = []
    url_serp = []
    delta_visit = []

    for item in response_.json()['data']:
        queries.append(item['dimensions'][0]['name'])
        url_serp.append(item['dimensions'][1]['name'])
        delta_visit.append(item['metrics']['b'][0]-item['metrics']['a'][0])

    queries_info = {
        'Запросы': queries,
        'URL перехода выдачи': url_serp,
        'Разница переходов': delta_visit
    }

    queries_df = pd.DataFrame(queries_info)
    print(url)
    result_queries_df = queries_df.loc[queries_df['Разница переходов'] < -5].sort_values(by=['Разница переходов'])
    display(result_queries_df)
    sub_url = url.replace('https://eldorado.ru', '')
    name_file = re.sub(r'(/)(.+)(/)(.+)(/)', r'\4', sub_url).replace('/', '_').replace('?', '').replace('.php', '')

    result_queries_df.to_csv("files/"+name_file+".csv", sep='\t')
    
    data = '\n'.join(result_queries_df['Запросы'].tolist())
    
    display(HTML('<a href="http://194.58.102.17:8888/edit/Reports/files/'+name_file+'.csv">Файл с запросами</a>'))
    print('http://194.58.102.17:8888/edit/Reports/files/'+name_file+'.csv')
    if data != '':
        clust_df = getJMclust(data)
        clust_df.to_excel("files/"+name_file+"_jmclust.xlsx")
        display(HTML('<a href="http://194.58.102.17:8888/edit/Reports/files/'+name_file+'_jmclust.xlsx">Файл с частотностью и кластеризацией запросов</a>'))
        print('http://194.58.102.17:8888/edit/Reports/files/'+name_file+'_jmclust.xlsx')
    print('---------------------------------------------------------')
    print('\n')


https://eldorado.ru/cat/detail/


,Запросы,URL перехода выдачи,Разница переходов
1,xbox series s,https://eldorado.ru/cat/detail/igrovaya-prista...,-383.0
0,xbox series s купить,https://eldorado.ru/cat/detail/igrovaya-prista...,-371.0
2,ps5 купить,https://eldorado.ru/cat/detail/igrovaya-prista...,-159.0
3,usb флеш-накопитель sandisk ultra flair usb 3....,https://eldorado.ru/cat/detail/usb-fleshka-san...,-131.0
7,электрическая зубная щетка braun oral-b vitali...,https://eldorado.ru/cat/detail/elektricheskaya...,-125.0
8,яндекс станция мини купить,https://eldorado.ru/cat/detail/umnaya-kolonka-...,-123.0
5,ps5 купить,https://eldorado.ru/cat/detail/igrovaya-prista...,-108.0
6,playstation 5 купить,https://eldorado.ru/cat/detail/igrovaya-prista...,-107.0
11,xbox series x,https://eldorado.ru/cat/detail/igrovaya-prista...,-87.0
12,самсунг а51,https://eldorado.ru/cat/detail/smartfon-samsun...,-86.0


http://194.58.102.17:8888/edit/Reports/files/detail.csv


http://194.58.102.17:8888/edit/Reports/files/detail_jmclust.xlsx
---------------------------------------------------------


https://eldorado.ru/info/shops/


,Запросы,URL перехода выдачи,Разница переходов


...

http://194.58.102.17:8888/edit/Reports/files/gazovye-plity.csv


http://194.58.102.17:8888/edit/Reports/files/gazovye-plity_jmclust.xlsx
---------------------------------------------------------


https://eldorado.ru/c/elektricheskie-plity/


,Запросы,URL перехода выдачи,Разница переходов
1,электрическая плита с духовкой,https://eldorado.ru/c/elektricheskie-plity/,-30.0
0,плита электрическая,https://eldorado.ru/c/elektricheskie-plity/,-28.0
6,электроплита 4-х конфорочная с духовкой купить,https://eldorado.ru/c/elektricheskie-plity/f/s...,-27.0
2,электроплита,https://eldorado.ru/c/elektricheskie-plity/,-18.0
5,купить электроплиту,https://eldorado.ru/c/elektricheskie-plity/,-18.0
8,купить электрическую плиту,https://eldorado.ru/c/elektricheskie-plity/,-10.0
10,электроплиты,https://eldorado.ru/c/elektricheskie-plity/,-9.0
3,электрическая плита,https://eldorado.ru/c/elektricheskie-plity/,-7.0


http://194.58.102.17:8888/edit/Reports/files/elektricheskie-plity.csv


http://194.58.102.17:8888/edit/Reports/files/elektricheskie-plity_jmclust.xlsx
---------------------------------------------------------




In [ ]:
df = pd.read_excel('files/elektricheskie-plity_jmclust.xlsx', na_values='0')
df_support = df.loc[:, ['key', 'url']]
df_support.rename(columns = {'key':'Запросы'}, inplace = True)
summary_df = result_queries_df.merge(df_support, on='Запросы', how='left')
summary_df['url'].fillna(summary_df['URL перехода выдачи'], inplace=True)
df_to_jm = summary_df.loc[:,['url', 'Запросы']]

url_list = df_to_jm['url'].tolist()
query_list = df_to_jm['Запросы'].tolist()
to_jm_list = []

for url, query in zip(url_list, query_list):
    to_jm_list.append(url+'\t'+query)

ta_data = '\n'.join(to_jm_list)


In [ ]:
_url_ = 'https://api.just-magic.org/api_v1.php'

request_ = {
    'action': 'put_task',
    'apikey': jm_api,
    'task': 'txt_anlz',
    'ya_lr': 213,
    'data': ta_data,
    'f_per': 1
    }

response = requests.post(_url_, data=request_)
response.json()


In [ ]:
_request_ = {
    'action': 'get_task',
    'apikey': jm_api,
    'tid': response.json()['tid'],
    'mode': 'csv'
}
response_ = requests.post(_url_, data=_request_)

with open('result_ta.csv.gz', 'wb') as file:
    file.write(response_.content)

with gzip.open('result_ta.csv.gz', 'rt') as f:
    data = f.read()
    with open('result_ta.csv', 'wt') as f:
        f.write(data)


In [ ]:
df = pd.read_excel('files/elektricheskie-plity_jmclust.xlsx', na_values='0')
df_support = df.loc[:, ['key', '!частот. !WordStat', 'url']]
df_support.rename(columns = {'key':'Запросы'}, inplace = True)
summary_df = result_queries_df.merge(df_support, on='Запросы', how='left')
summary_df['url'].fillna(summary_df['URL перехода выдачи'], inplace=True)
df_to_jm = summary_df.loc[:,['url', '!частот. !WordStat', 'Запросы']]
df_to_jm.sort_values(by=['!частот. !WordStat'], ascending=False, inplace=True)
df_to_jm.sort_values(by=['url'], inplace=True)
df_to_jm.drop_duplicates(subset=['url'], keep='first', inplace=True)
query_list = df_to_jm['Запросы'].tolist()
lsi_data = '\n'.join(query_list)
lsi_data


In [ ]:
_url_ = 'https://api.just-magic.org/api_v1.php'

request_ = {
    'action': 'put_task',
    'apikey': jm_api,
    'task': 'aqua_gen',
    'data': lsi_data,
    }

response = requests.post(_url_, data=request_)
response.json()

In [ ]:
for task in response.json()['tid']:
    _request_ = {
        'action': 'get_task',
        'apikey': jm_api,
        'tid': task,
        'mode': 'csv'
    }
    while True:
        response_ = requests.post(_url_, data=_request_)
        header = 'Content-disposition'
        if header in response_.headers:
            with open('result'+task+'.csv.gz', 'wb') as file:
                file.write(response_.content)

            df = pd.read_csv('result'+task+'.csv.gz', compression='gzip', sep='\t')
            df.to_excel('files/'+response.json()['tid'][task].replace(' ', '_')+'.xlsx')
            print('http://194.58.102.17:8888/edit/Reports/files/'+response.json()['tid'][task].replace(' ', '_')+'.xlsx')
            break